# Testing the Infant filter

Will try to load the Infant filter and test it by parsing a tar of alerts.

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

First step is to load the filter, using the path specified in the `setup.py` file in the repository. 


In [ ]:
from ampel.contrib.weizmann.t0.InfantFilter import InfantFilter

The full behaviour of a T0 unit is controlled by a set of parameters provided as `run_config`. We here directly specify these as a dictionary. When implemented for a live channel these constitue one entry to the `channel.json` main configuration file.

In [ ]:
runConfig = {
                    "MIN_NDET": 1,
                    "MIN_TSPAN": 0.02,
                    "MAX_TSPAN": 4.5,
                    "MAX_TUL": 2.5,
                    "MIN_RB": 0.3,
                    "MIN_DRB": 0.3,
                    "MIN_FWHM": 0.5,
                    "MAX_FWHM": 5.0,
                    "MAX_ELONG": 100,
                    "MAX_MAGDIFF": 0.75,
                    "MAX_NBAD": 5,
                    "MIN_DIST_TO_SSO": 20,
                    "MIN_GAL_LAT": 14,
                    "GAIA_RS": 40,
                    "GAIA_PM_SIGNIF": 3,
                    "GAIA_PLX_SIGNIF": 3,
                    "GAIA_VETO_GMAG_MIN": 9,
                    "GAIA_VETO_GMAG_MAX": 20,
                    "GAIA_EXCESSNOISE_SIG_MAX": 999,
                    "PS1_SGVETO_RAD": 2,
                    "PS1_SGVETO_SGTH": 0.76,
                    "PS1_CONFUSION_RAD": 3,
                    "PS1_CONFUSION_SG_TOL": 0.1
                    }


In [ ]:
baseModel_runConfig = InfantFilter.RunConfig(**runConfig)

In [ ]:
import os
import urllib.request

small_test_tar_url = 'https://ztf.uw.edu/alerts/public/ztf_public_20181129.tar.gz'
small_test_tar_path = '/home/jnordin/data/ztfalerts/ztf_public_20181129.tar.gz'

if not os.path.isfile(small_test_tar_path):
    print('Downloading tar')
    urllib.request.urlretrieve(small_test_tar_url, small_test_tar_path)


The Decent does assume the catSHTM server to pe present. Have to write a nice workaraound for test cases when it is not.

In [ ]:
on_match_t2_units = []
baseConfig = {'catsHTM.default':None}

In [ ]:
from ampel.ztf.pipeline.t0.DevAlertProcessor import DevAlertProcessor

my_filter = InfantFilter(on_match_t2_units, base_config=baseConfig, run_config=baseModel_runConfig, logger=logger)
dap = DevAlertProcessor(my_filter, use_dev_alerts=True)

In [ ]:
import time

print ("processing alerts from %s" % small_test_tar_path)
start = time.time()
nproc = dap.process_tar(small_test_tar_path, iter_max=200)
end = time.time()
print ("processed %d alerts in %.2e sec"%(nproc, end-start))

In [ ]:
n_good, n_bad = len(dap.get_accepted_alerts()), len(dap.get_rejected_alerts())
print ("%d alerts accepted by the filter (%.2f perc)"%(n_good, 100*n_good/nproc))
print ("%d alerts rejected by the filter (%.2f perc)"%(n_bad, 100*n_bad/nproc))

In [ ]:
from ampel.ztf.view.AmpelAlertPlotter import AmpelAlertPlotter
from random import choice

accepted = dap.get_accepted_alerts()
accepted_plot = AmpelAlertPlotter(interactive=True)

In [ ]:
accepted_plot.summary_plot(choice(accepted))